# Gini


In [627]:
%load_ext autoreload
%autoreload 2
import altair as alt
import fetch_data as fd
import pandas as pd
import numpy as np
import os

SHOULD_LOG = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [628]:
city_info = fd.get_city_info()
city_info.head()

,regional key,city,sq km,population
0,11000000,Berlin,891.12,3685265
1,02000000,Hamburg,755.09,1862565
2,09162000,München,310.70,1505005
3,05315000,Köln,405.02,1024621
4,06412000,Frankfurt am Main,248.31,756021


In [629]:
YEARS = [i for i in range(2016, 2025)]
df_dict = fd.get_dfs(YEARS)
df = pd.concat(df_dict.values(), ignore_index=True)

## Data Cleaning


Severity of accidents.


In [630]:
df = pd.get_dummies(df, columns=["UKATEGORIE"], prefix="inj", dtype=int)
df.rename(
    columns={
        "inj_3": "inj_light",
        "inj_2": "inj_serious",
        "inj_1": "inj_fatal",
    },
    inplace=True,
)

Group on the `Community_key` (city) and `UJAHR` the year. Additionally, aggregate metrics we care about.


In [631]:
agg_methods = {
    "inj_light": "sum",
    "inj_serious": "sum",
    "inj_fatal": "sum",
    "IstFuss": "sum",
    "IstRad": "sum",
    "ULAND": "first",
}


df_grouped = df.groupby(["Community_key", "UJAHR"]).agg(agg_methods).reset_index()

Perform an inner join on `"regional key"` with `df_grouped` and `city_info`.


In [632]:
df_grouped.rename(columns={"Community_key": "regional key"}, inplace=True)
df_merged = df_grouped.merge(city_info, on="regional key", how="inner")

Calculate some metrics we care about.


In [633]:
# Calculate the total personal injury accidents
df_merged["inj_total"] = (
    df_merged["inj_light"] + df_merged["inj_serious"] + df_merged["inj_fatal"]
)

In [634]:
df_merged[df_merged["UJAHR"] == 2024].head()

,regional key,UJAHR,inj_light,inj_serious,inj_fatal,IstFuss,IstRad,ULAND,city,sq km,population,inj_total
8,01001000,2024,325,22,2,34,151,01,Flensburg,56.73,96326,349
17,01002000,2024,930,93,4,102,514,01,Kiel,118.65,252668,1027
26,01003000,2024,1008,109,1,104,611,01,Lübeck,214.19,216889,1118
35,01004000,2024,329,33,0,32,136,01,Neumünster,71.66,79809,362
44,01051011,2024,51,9,0,5,25,01,Brunsbüttel,65.21,12692,60


In [635]:
df_merged[df_merged["UJAHR"] == 2024].describe()

,UJAHR,inj_light,inj_serious,inj_fatal,IstFuss,IstRad,sq km,population,inj_total
count,2050.0,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2.050000e+03,2050.000000
mean,2024.0,84.535610,13.824390,0.693659,10.487317,34.477073,73.501815,2.998465e+04,99.053659
std,0.0,349.707445,46.972889,1.799031,53.794324,154.442712,63.039939,1.130745e+05,397.130578
min,2024.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.710000,4.470000e+02,1.000000
25%,2024.0,12.000000,3.000000,0.000000,1.000000,3.000000,32.645000,6.264250e+03,17.000000
50%,2024.0,29.000000,7.000000,0.000000,3.000000,9.000000,59.885000,1.275250e+04,37.000000
75%,2024.0,65.000000,13.000000,1.000000,7.000000,26.000000,93.605000,2.517450e+04,80.000000
max,2024.0,10739.000000,1641.000000,48.000000,1712.000000,4452.000000,891.120000,3.685265e+06,12428.000000


## Gini

We calculate the Gini index. Normally, the Gini index is used as a measure for wealth disparity. Here, we seek to use it as a metric to inform about the relative share of fatalities between cities. In this context, a Gini coefficient of 0 would mean that fatalities are equally distributed between cities while a Gini coefficient of 1 would mean that the fatalities are are concentrated in one city.


In [636]:
pop_label = "population"
val_label = "inj_fatal"


# Gini Calculation Lifted from Lucas. Retrieved from:
# https://gitlab.com/ComputationalScience/overdose-da
def calc_gini(
    df: pd.DataFrame, pop_label: str = pop_label, val_label: str = val_label
) -> float:
    pop = df.sort_values(pop_label, ascending=True)[pop_label].sum()
    val = df.sort_values(pop_label, ascending=True)[val_label].sum()

    """
        From Lucas:
        gini = 1 - 2 * np.trapezoid(
            [
                data.sort_values(by=["Population"], ascending=False)[:i]["Deaths"].sum()
                / deaths
                for i in range(
                    len(data.sort_values(by=["Population"], ascending=False)["Deaths"])
                )
            ],
            [
                data.sort_values(by=["Population"], ascending=False)[:i]["Population"]
                .astype(int)
                .sum()
                / population
                for i in range(
                    len(data.sort_values(by=["Population"], ascending=False)["Population"])
                )
            ],
        )
    """
    return 1 - np.trapezoid(
        [
            df.sort_values(pop_label, ascending=True)[:i][val_label].sum() / val
            for i in range(len(df.sort_values(pop_label, ascending=True)[val_label]))
        ],
        [
            df.sort_values(pop_label, ascending=True)[:i][pop_label].sum() / pop
            for i in range(len(df.sort_values(pop_label, ascending=True)[pop_label]))
        ],
    )

In [646]:
gini_arr: list[float] = []

for year in YEARS:
    df_yr = df_merged[df_merged["UJAHR"] == year][[pop_label, val_label]]

    if SHOULD_LOG:
        pop = df_yr.sort_values(pop_label, ascending=False)[pop_label].sum()
        val = df_yr.sort_values(pop_label, ascending=False)[val_label].sum()

        print(f"Year {year}:")
        print(f"  {pop_label}: {pop:,},")
        print(f"  {val_label}: {val:,}")

    gini = calc_gini(df_yr)
    gini_arr.append(gini)

In [647]:
if SHOULD_LOG:
    print("\n".join([f"{y}: {g:0.3f}" for y, g in zip(YEARS, gini_arr)]))

In [648]:
title = " ".join(val_label.split("_")).title() if "_" in val_label else val_label

chart = (
    alt.Chart(pd.DataFrame({"Year": YEARS, "Gini": gini_arr}))
    .mark_line(point=True)
    .properties(
        title=f"Gini Coefficient of {title} per Year",
        width=600,
        height=400,
    )
    .encode(x="Year:O", y="Gini:Q")
    .interactive()
)

chart.save(os.path.join("img", f"gini_index_{title.replace(' ', '_').lower()}.png"))
chart.show()

alt.Chart(...)

Lets now try to compare the gini index per state.


In [649]:
land_LUT: dict[int, str] = {
    1: "Schleswig-Holstein",
    2: "Hamburg",
    3: "Niedersachsen",
    4: "Bremen",
    5: "Nordrhein-Westfalen",  # data as from 2019
    6: "Hessen",
    7: "Rheinland-Pfalz",  # data as from 2017
    8: "Baden-Württemberg",
    9: "Bayern",
    10: "Saarland",  # data as from 2017
    11: "Berlin",  # data as from 2018
    12: "Brandenburg",  # data as from 2017
    13: "Mecklenburg-Vorpommern",  # data as from 2020
    14: "Sachsen",
    15: "Sachsen-Anhalt",
    16: "Thüringen",  # data as from 2019
}

In [650]:
gini_arr: list[dict[str, float | str | int]] = []

for year in YEARS:
    # group by state for this year
    df_yr = df_merged[df_merged["UJAHR"] == year][[pop_label, val_label, "ULAND"]]
    if SHOULD_LOG:
        print(f"Year {year}:")

    for l_id, l_name in land_LUT.items():
        # We skip Berlin (11) and Hamburg (2) since they are city-states. Thus, the Gini
        # is 1 and not informative
        if l_id in [2, 11]:
            continue

        df_land = df_yr[df_yr["ULAND"] == f"{l_id:02}"]

        # Skip if we have no data for this land in this year
        if df_land.empty:
            continue

        if SHOULD_LOG:
            pop = df_land[pop_label].sum()
            val = df_land[val_label].sum()

            print(f"  Land {l_name}:")
            print(f"    {pop_label}: {pop:,}")
            print(f"    {val_label}: {val:,}")

        gini = calc_gini(df_land)
        gini_arr.append(
            {
                "Year": year,
                "Land": l_name,
                "Gini": gini,
            }
        )

In [651]:
df_gini = pd.DataFrame(gini_arr)
df_gini.describe()

,Year,Gini
count,111.000000,111.000000
mean,2020.387387,0.567920
std,2.468607,0.144911
min,2016.000000,0.432859
25%,2018.000000,0.478230
50%,2020.000000,0.526507
75%,2022.500000,0.569804
max,2024.000000,1.000000


In [652]:
title = " ".join(val_label.split("_")).title() if "_" in val_label else val_label

chart = (
    alt.Chart(df_gini)
    .mark_line(point=True)
    .encode(
        x=alt.X("Year:O", title="Year"),
        y=alt.Y("Gini:Q", title="Gini Coefficient"),
        color=alt.Color("Land:N", title="State"),
        tooltip=[
            "Land",
            alt.Tooltip("Year:O", title="Year"),
            alt.Tooltip("Gini:Q", format=".3f"),
        ],
    )
    .properties(
        title=f"Gini Coefficient of {title} by State per Year",
        width=800,
        height=400,
    )
    .interactive()
)

chart.save(
    os.path.join("img", f"gini_index-{title.replace(' ', '_').lower()}_land.png")
)
chart.show()

alt.Chart(...)